In [1]:
cd drive/My\ Drive/concrete_crack

/content/drive/My Drive/concrete_crack


In [2]:
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, accuracy_score, classification_report
import seaborn as sns
from sklearn.metrics import mean_squared_error as mse
import os

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
%matplotlib inline

In [4]:
cd model7/

/content/drive/My Drive/concrete_crack/model7


# Loading the Pre-trained Encoder and Decoder Models

In [5]:
encoder = load_model('encoder.h5')
decoder = load_model('decoder.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
encoder.summary()

In [0]:
decoder.summary()

In [0]:
cd ..

In [0]:
cd data_64/

# Data Selection

In [0]:
positive_data = np.load('X_test_positive_imbalanced.npy', allow_pickle=True)
negative_data = np.load('X_test_negative.npy', allow_pickle=True)
positive_index = np.random.permutation(len(positive_data))
positive_data = positive_data[positive_index]
negative_index = np.random.permutation(len(negative_data))
negative_data = negative_data[negative_index]
demo_positive_data = positive_data[:10]
demo_negative_data = negative_data[:1000]
demo_data = np.concatenate([demo_negative_data, demo_positive_data])

In [0]:
positive_label = np.array([1]*10)
negative_label = np.array([0]*1000)
y_true = np.concatenate([negative_label, positive_label])

In [0]:
encoded_data = encoder.predict(demo_data)
decoded_data = decoder.predict(encoded_data)

# Computing the MSE between original and reconstructed images

In [0]:
def compute_mse_scores(data1, data2):
  error = []
  for i,j in zip(data1, data2):
    error.append(mse(i.flatten(), j.flatten()))
  return np.array(error)

In [0]:
mse_score = compute_mse_scores(demo_data, decoded_data)

In [0]:
mse_normal = mse_score[:1000]
mse_anomalous = mse_score[1000:]
plt.figure(figsize = (10, 6))
sns.distplot(mse_normal, kde = 'False', label = 'Normal')
sns.distplot(mse_anomalous, kde = 'False', label = 'Anomalous')
plt.xlabel('MSE Score')
plt.ylabel('Count')
plt.title('MSE Score Distribution of Normal and Anomalous Demo Samples')
plt.legend()
plt.show()

# AUC Computation based on MSE Scores

In [0]:
fpr, tpr, thr = roc_curve(y_true, mse_score)
roc_auc = auc(fpr, tpr)
plt.figure(figsize = (10, 6))
plt.plot(fpr, tpr, label = 'Model VII AUC on Demo Data : %0.4f'%roc_auc)
plt.plot([0,1], [0,1], 'r--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Model VII AUC for Demo Data based on MSE Score')
plt.legend()
plt.show()

# Precision and Recall at Different MSE Thresholds

In [0]:
precision, recall, threshold = precision_recall_curve(y_true, mse_score)
plt.figure(figsize=(10, 6))
plt.plot(threshold, precision[1:], label = 'Precision')
plt.plot(threshold, recall[1:], label = 'Recall')
plt.xlabel('Threshold')
plt.ylabel('Precision and Recall')
plt.title('Precision and Recall at Different Thresholds for Model VII based on MSE Scores')
plt.legend()
plt.show()

# Testing the MSE Threshold

In [0]:
threshold_level = float(input('Enter a threshold : '))
df = pd.DataFrame({'Label' : y_true, 'Score' : mse_score})
df = df.sample(frac = 1).reset_index(drop = True)
normal_df = df.loc[df['Label'] == 0]
anomalous_df = df.loc[df['Label'] == 1]
plt.figure(figsize = (10,6))
plt.scatter(normal_df.index, normal_df['Score'], alpha = 0.5, label = 'normal')
plt.scatter(anomalous_df.index, anomalous_df['Score'], alpha = 0.5, label = 'Anomalous')
plt.hlines(0.007, plt.xlim()[0], plt.xlim()[1], colors='r', label = 'Threshold')
plt.xlabel('Index Position of the Data Point')
plt.ylabel('MSE Score')
plt.title('Testing the Classification Threshold at MSE : %.4f'%(threshold_level))
plt.legend()
plt.show()

# Checking the Model Accuracy

In [0]:
predicted_label = [1 if e > threshold_level else 0 for e in mse_score]
conf_mat = confusion_matrix(y_true, predicted_label)
df = pd.DataFrame(conf_mat, index = ['Normal', 'Anomalous'], columns = ['Normal', 'Anomalous'])
plt.figure(figsize = (10, 6))
sns.heatmap(df, annot = True, fmt = 'd')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Confusion Matrix of Model VII with an accuracy of %.4f'%accuracy_score(y_true, predicted_label))
plt.show()
print('----------------------------------------------------------')
print('                   Classification Report                  ')
print('----------------------------------------------------------')
print(classification_report(y_true, predicted_label))